In [1]:
# importeren packages
import pandas as pd
import numpy as np
#conda install -c anaconda py-xgboost #installeer XGBoost
import xgboost
import pickle
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
#conda install -c anaconda py-xgboost #installeer XGBoost
import xgboost
#from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold, cross_val_score

In [2]:
# setten source path: later moet alle brondata naar datalake
source_path ="P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"
# alle tussentijdse data voorlopig in dezelfde map op een schijf opslaan in
save_path = "P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"

In [3]:
# inladen bestand met alle buurten en onafhankelijke variabelen als all
buurtendata = pd.read_csv(source_path + 'buurten.csv', encoding='utf-16')
buurtendata.rename(columns={"RLBBEV18":"RLBBEV",'KL18': "KL", 
               'RLBRMTR18': 'RLBRMTR', 'RLBWON18':'RLBWON' , 'RLBBEV18':'RLBBEV', 'RLBVRZ18':'RLBVRZ', 'RLBVEI18': 'RLBVEI', 'RLBFYS18': 'RLBFYS',}, inplace=True) 

In [4]:
# inladen bestand met scores als sc
sc = pd.read_excel(source_path + 'buurten voor experts 2e ronde.xlsx', sheet_name='buurten')
# alleen selecteren kolommen voor scores en oordelen
sc = sc[['buurtcode','EE','TK']]

In [5]:
# merge op buurtcode (die van 2019)
df = pd.merge(sc, buurtendata, on='buurtcode')

In [6]:
# minus de kolommen die niet in de 2016 dataset aanwezig zijn
feature_cols = ['AantalCafe3', 'woxdelta5jr', 'woxm2', 'woxww',
       'woxdelta5jrGem', 'wox', 'RLBBEV',
       'RLBVRZ', 'AantalLevensm3', 'woxm2WG', 'aand1945-1970',
       'woxdelta5jrWGabs', 'woxdelta1jrGem', 'AantalPodiumk10',
       'wv2000-2010', 'AantalBios5']
       
X = df[feature_cols]
y = df['EE']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

In [9]:
# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)

In [10]:
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

train_accuracy = accuracy_score(y_train,y_train_pred)
test_accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)
print(train_accuracy, '\n',test_accuracy,'\n', cm)

0.9920948616600791 
 0.9176470588235294 
 [[38  1  0]
 [ 3 25  2]
 [ 1  0 15]]


In [11]:
fi = pd.DataFrame(model.feature_importances_,X_train.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
woxm2,0.223192,woxm2
woxww,0.152606,woxww
woxm2WG,0.102962,woxm2WG
woxdelta1jrGem,0.082279,woxdelta1jrGem
AantalPodiumk10,0.078607,AantalPodiumk10
wox,0.073167,wox
woxdelta5jr,0.061225,woxdelta5jr
woxdelta5jrWGabs,0.036442,woxdelta5jrWGabs
wv2000-2010,0.032491,wv2000-2010
RLBBEV,0.029391,RLBBEV


In [ ]:
# Het opslaan van het model
file_name = "models/final_model_EE_wox.pkl"
pickle.dump(model, open(file_name, "wb"))

## Volledige dataset voorspelling
Hier testen we de accuracy als wij een getraind model gebruiken om de volledige dataset te voorspellen.

In [30]:
# Wel enkel de kolommen nodig waarop ons model is getraind
X_pred = buurtendata[feature_cols]
X_pred = pd.DataFrame(X_pred)
X_pred.columns = feature_cols

In [31]:
# Dit is het erwin model dus deze waarde voorspellen wij
X_pred['EE_preds'] = model.predict(X_pred)
X_pred['buurtcode'] = buurtendata['buurtcode']

NameError: name 'model' is not defined

In [16]:
# Hier koppelen we de 'echte' waardes hieraan om de accuracy te voorspellen.
X_pred = X_pred.merge(sc, how='left', on ='buurtcode')
temp = X_pred[X_pred['EE'].notnull()][['EE', 'EE_preds']]
accuracy_score(temp['EE_preds'].values,temp['EE'].values)


## Thomas model

In [22]:
# Hier hebben wij een dataset nodig met de labels van Thomas. 
X = df[feature_cols]
y = df['TK']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2) # dmv de seed is de verdeling van de items hetzelfde

In [23]:
# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)

In [24]:
# voorspellen en berekenen van de accuracy
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

train_accuracy = accuracy_score(y_train,y_train_pred)
test_accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)
print(train_accuracy, '\n',test_accuracy,'\n', cm)

1.0 
 0.9058823529411765 
 [[38  1  1]
 [ 1 11  4]
 [ 1  0 28]]


In [25]:
# Opslaan van het model
file_name = "models/final_model_TK_wox.pkl"
pickle.dump(model, open(file_name, "wb"))

## De modellen zonder WOX
Hierbij wordt gebruik gemaakt van andere kolommmen. 

In [26]:
feature_cols_nw = ['inw', 'bevdh', 'oadh', 'hh', 'inw65', 'KL', 
               'RLBRMTR', 'RLBWON', 'RLBBEV', 'RLBVRZ', 'RLBVEI', 'RLBFYS', 
               'AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3','AantalRest3', 'AantalKinderopvang1', 
               'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 
               'hh19inkLaag', 'hh30inkLaag', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog', 
               'aandeelMH19', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 'banen'] 

In [27]:
# Eerst thomas labels
X = df[feature_cols_nw]
y = df['TK']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

In [28]:
# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)

In [29]:
# Voorspellen en beoordelen van de resultaten
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

train_accuracy = accuracy_score(y_train,y_train_pred)
test_accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)
print(train_accuracy, '\n',test_accuracy,'\n', cm)

1.0 
 0.8705882352941177 
 [[38  1  1]
 [ 4  8  4]
 [ 0  1 28]]


In [30]:
# Model opslaan
file_name = "models/final_model_TK_nowox.pkl"
pickle.dump(model, open(file_name, "wb"))

In [31]:
fi = pd.DataFrame(model.feature_importances_,X_train.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
AantalCafe3,0.099049,AantalCafe3
AantalBios5,0.094162,AantalBios5
fiets,0.067854,fiets
auto60,0.053232,auto60
AantalPodiumk10,0.053218,AantalPodiumk10
RLBBEV,0.047427,RLBBEV
RLBVRZ,0.043478,RLBVRZ
AantalLevensm3,0.040715,AantalLevensm3
ov60,0.033228,ov60
AantalMusea10,0.032168,AantalMusea10


## Het model zonder Wox van Erwin

In [32]:
X = df[feature_cols_nw]
y = df['EE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

In [48]:
# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)
# en fit op data
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)

In [49]:
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

train_accuracy = accuracy_score(y_train,y_train_pred)
test_accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)
print(train_accuracy, '\n',test_accuracy,'\n', cm)

1.0 
 0.8117647058823529 
 [[34  5  0]
 [ 3 22  5]
 [ 1  2 13]]


In [36]:
file_name = "models/final_model_EE_nowox.pkl"
pickle.dump(model, open(file_name, "wb"))

# Voorspellingen gebruiken/ testen

In [15]:
# laad model
model_2019_EE = pickle.load(open("../final_models/final_model_EE_wox.pkl", "rb"))
model_2019_TK = pickle.load(open("../final_models/final_model_TK_wox.pkl", "rb"))
model_2019_EE_nw = pickle.load(open("../final_models/final_model_EE_nowox.pkl", "rb"))
model_2019_TK_nw = pickle.load(open("../final_models/final_model_TK_nowox.pkl", "rb"))

In [19]:
# de preditie dataframes maken:
X_pred_w = buurtendata[feature_cols]
X_pred_w = pd.DataFrame(X_pred_w)
X_pred_w.columns = feature_cols

# No wox:
feature_cols_nw = ['inw', 'bevdh', 'oadh', 'hh', 'inw65', 'KL', 
               'RLBRMTR', 'RLBWON', 'RLBBEV', 'RLBVRZ', 'RLBVEI', 'RLBFYS', 
               'AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3','AantalRest3', 'AantalKinderopvang1', 
               'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 
               'hh19inkLaag', 'hh30inkLaag', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog', 
               'aandeelMH19', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 'banen'] 


X_pred_nw = buurtendata[feature_cols_nw]
X_pred_nw = pd.DataFrame(X_pred_nw)
X_pred_nw.columns = feature_cols_nw

In [20]:
predictions_EE = model_2019_EE.predict(X_pred_w)
predictions_TK = model_2019_TK.predict(X_pred_w)
predictions_EE_nw = model_2019_EE_nw.predict(X_pred_nw)
predictions_TK_nw = model_2019_TK_nw.predict(X_pred_nw)

In [21]:
buurtendata['voorspellingen_TK'] = predictions_TK
buurtendata['voorspellingen_EE'] = predictions_EE
buurtendata['voorspellingen_TK_nw'] = predictions_TK_nw
buurtendata['voorspellingen_EE_nw'] = predictions_EE_nw

In [22]:
len(buurtendata)

13632

In [23]:
df_final = pd.merge(buurtendata,sc,how='left', on='buurtcode')

In [24]:
df_final.shape

(13632, 119)

In [25]:
df_temp = df_final[df_final['EE'].notnull()]
print("accuracy score EE normal ", accuracy_score(df_temp['EE'], df_temp['voorspellingen_EE']))
print("accuracy score TK normal ", accuracy_score(df_temp['TK'], df_temp['voorspellingen_TK']))
print("accuracy score EE no wox ", accuracy_score(df_temp['EE'], df_temp['voorspellingen_EE_nw']))
print("accuracy score TK no wox ", accuracy_score(df_temp['TK'], df_temp['voorspellingen_TK_nw']))

accuracy score EE normal  0.9733727810650887
accuracy score TK normal  0.9763313609467456
accuracy score EE no wox  0.9526627218934911
accuracy score TK no wox  0.9674556213017751


In [26]:
df_final[['buurtcode', 'EE', 'TK', 'voorspellingen_EE', 'voorspellingen_TK','voorspellingen_EE_nw', 'voorspellingen_TK_nw']].head()

,buurtcode,EE,TK,voorspellingen_EE,voorspellingen_TK,voorspellingen_EE_nw,voorspellingen_TK_nw
0,30000,NaN,NaN,c,c,c,c
1,30001,NaN,NaN,c,c,b,c
2,30002,NaN,NaN,c,c,c,c
3,30007,NaN,NaN,c,c,a,c
4,30008,NaN,NaN,c,c,c,c


In [27]:
df_final['wox_info'] = df_final['wox'].notnull()
df_final['EE_Choice'] = df_final.apply(lambda x: x['voorspellingen_EE'] if x['wox_info'] else x['voorspellingen_EE_nw'], axis=1)
df_final['TK_Choice'] = df_final.apply(lambda x: x['voorspellingen_TK'] if x['wox_info'] else x['voorspellingen_TK_nw'], axis=1)

In [28]:
sum(df_final['wox'].notnull())

6737

In [29]:
df_final[['buurtcode', 'EE', 'TK', 'voorspellingen_EE', 'voorspellingen_TK','voorspellingen_EE_nw', 'voorspellingen_TK_nw', 'wox_info', 'EE_Choice', 'TK_Choice']].to_csv('../predictions buurtscores/predictions_final.csv')